# Multi-Frame CRNN Training Walkthrough

This notebook breaks down the training pipeline for a Multi-Frame CRNN model used for text recognition from video tracks. We will go through each component step-by-step.

## 1. Imports and Setup
First, we import necessary libraries and set a random seed for reproducibility.

In [ ]:
import os
import glob
import json
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from tqdm import tqdm
from torch.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🔒 Đã cố định Seed: {seed}")

## 2. Configuration
Define all hyperparameters and constants in a Config class.

In [ ]:
class Config:
    DATA_ROOT = "/kaggle/input/icpr2026/train"
    IMG_HEIGHT = 32
    IMG_WIDTH = 128
    CHARS = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ-"
    BATCH_SIZE = 64
    LEARNING_RATE = 0.001
    EPOCHS = 50
    SEED = 42
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    CHAR2IDX = {char: idx + 1 for idx, char in enumerate(CHARS)}
    IDX2CHAR = {idx + 1: char for idx, char in enumerate(CHARS)}
    NUM_CLASSES = len(CHARS) + 1
    VAL_SPLIT_FILE = "val_tracks.json"
    TEST_SPLIT_FILE = "test_tracks.json"
    VAL_SIZE = 2000
    TEST_SIZE = 2000

## 3. Data Augmentation
Define transformations for training (with augmentation) and validation/testing.

In [ ]:
def get_train_transforms():
    return A.Compose([
        A.Resize(height=Config.IMG_HEIGHT, width=Config.IMG_WIDTH),

        # FIX: cval -> fill
        A.Affine(scale=(0.95, 1.05), translate_percent=(0.05, 0.05), rotate=(-5, 5), p=0.5, fill=128),
        A.Perspective(scale=(0.02, 0.05), p=0.3),

        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, p=0.3),

        A.CoarseDropout(num_holes_range=(1, 3), hole_height_range=(4, 8), hole_width_range=(4, 8), p=0.3),

        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2()
    ])

def get_degradation_transforms():
    return A.Compose([
        A.OneOf([
            A.GaussianBlur(blur_limit=(3, 7), p=1.0),
            A.MotionBlur(blur_limit=(3, 7), p=1.0),
            A.Defocus(radius=(1, 3), alias_blur=(0.1, 0.3), p=1.0),
        ], p=0.8),

        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
            A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1.0),
            A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=1.0),
        ], p=0.8),

        A.ImageCompression(quality_range=(10, 50), p=0.5),
        A.Downscale(scale_range=(0.25, 0.5), p=0.5),
    ])

def get_val_transforms():
    return A.Compose([
        A.Resize(height=Config.IMG_HEIGHT, width=Config.IMG_WIDTH),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2()
    ])

## 4. Dataset Class
Custom Dataset class to handle multi-frame loading and splitting.

In [ ]:
class AdvancedMultiFrameDataset(Dataset):
    def __init__(self, root_dir, mode='train'):
        self.mode = mode
        self.samples = []

        if mode == 'train':
            self.transform = get_train_transforms()
            self.degrade = get_degradation_transforms()
        else:
            self.transform = get_val_transforms()
            self.degrade = None

        print(f"[{mode.upper()}] Scanning: {root_dir}")
        abs_root = os.path.abspath(root_dir)
        search_path = os.path.join(abs_root, "**", "track_*")
        all_tracks = sorted(glob.glob(search_path, recursive=True))

        if not all_tracks:
            print("❌ LỖI: Không tìm thấy data.")
            return

        train_tracks = []
        val_tracks = []
        test_tracks = []

        # Check if split files exist
        val_exists = os.path.exists(Config.VAL_SPLIT_FILE)
        test_exists = os.path.exists(Config.TEST_SPLIT_FILE)

        val_ids = set()
        test_ids = set()

        if val_exists and test_exists:
            print(f"📂 Loading splits from '{Config.VAL_SPLIT_FILE}' and '{Config.TEST_SPLIT_FILE}'...")
            try:
                with open(Config.VAL_SPLIT_FILE, 'r') as f:
                    val_ids = set(json.load(f))
                with open(Config.TEST_SPLIT_FILE, 'r') as f:
                    test_ids = set(json.load(f))
            except:
                val_ids = set()
                test_ids = set()
                print("⚠️ Lỗi đọc file split, sẽ tạo lại.")

            for t in all_tracks:
                track_name = os.path.basename(t)
                if track_name in val_ids:
                    val_tracks.append(t)
                elif track_name in test_ids:
                    test_tracks.append(t)
                else:
                    train_tracks.append(t)

            # Nếu split không khớp, tạo lại
            if (not val_tracks or not test_tracks) and len(all_tracks) > 0:
                print("⚠️ File split không khớp với dữ liệu hiện tại. Chia lại...")
                val_ids = set()
                test_ids = set()

        if not val_ids or not test_ids:
            print(f"⚠️ Creating new split: {Config.VAL_SIZE} val, {Config.TEST_SIZE} test...")
            random.Random(Config.SEED).shuffle(all_tracks)

            # Chia: val 2000, test 2000, train còn lại
            val_tracks = all_tracks[:Config.VAL_SIZE]
            test_tracks = all_tracks[Config.VAL_SIZE:Config.VAL_SIZE + Config.TEST_SIZE]
            train_tracks = all_tracks[Config.VAL_SIZE + Config.TEST_SIZE:]

            # Lưu split files
            val_ids = [os.path.basename(t) for t in val_tracks]
            test_ids = [os.path.basename(t) for t in test_tracks]
            with open(Config.VAL_SPLIT_FILE, 'w') as f:
                json.dump(val_ids, f, indent=2)
            with open(Config.TEST_SPLIT_FILE, 'w') as f:
                json.dump(test_ids, f, indent=2)
            print(f"✅ Saved splits: val={len(val_tracks)}, test={len(test_tracks)}, train={len(train_tracks)}")

        if mode == 'train':
            selected_tracks = train_tracks
        elif mode == 'val':
            selected_tracks = val_tracks
        else:  # mode == 'test'
            selected_tracks = test_tracks
        print(f"[{mode.upper()}] Loaded {len(selected_tracks)} tracks.")

        for track_path in tqdm(selected_tracks, desc=f"Indexing {mode}"):
            json_path = os.path.join(track_path, "annotations.json")
            if not os.path.exists(json_path): continue
            try:
                with open(json_path, 'r') as f:
                    data = json.load(f)
                if isinstance(data, list): data = data[0]
                label = data.get('plate_text', data.get('license_plate', data.get('text', '')))
                if not label: continue

                lr_files = sorted(glob.glob(os.path.join(track_path, "lr-*.png")) + glob.glob(os.path.join(track_path, "lr-*.jpg")))
                hr_files = sorted(glob.glob(os.path.join(track_path, "hr-*.png")) + glob.glob(os.path.join(track_path, "hr-*.jpg")))

                if len(lr_files) > 0:
                    self.samples.append({
                        'lr_paths': lr_files,
                        'hr_paths': hr_files,
                        'label': label
                    })
            except: pass

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        item = self.samples[idx]
        label = item['label']

        use_hr = (self.mode == 'train') and (len(item['hr_paths']) > 0) and (random.random() < 0.5)

        if use_hr:
            img_paths = item['hr_paths']
            if len(img_paths) < 5: img_paths = img_paths + [img_paths[-1]] * (5 - len(img_paths))
            else: img_paths = img_paths[:5]

            images_list = []
            for p in img_paths:
                image = cv2.imread(p)
                if image is None: image = np.zeros((Config.IMG_HEIGHT, Config.IMG_WIDTH, 3), dtype=np.uint8)
                else: image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                if self.degrade:
                    image = self.degrade(image=image)['image']
                image = self.transform(image=image)['image']
                images_list.append(image)
        else:
            img_paths = item['lr_paths']
            if len(img_paths) < 5: img_paths = img_paths + [img_paths[-1]] * (5 - len(img_paths))
            else: img_paths = img_paths[:5]

            images_list = []
            for p in img_paths:
                image = cv2.imread(p)
                if image is None: image = np.zeros((Config.IMG_HEIGHT, Config.IMG_WIDTH, 3), dtype=np.uint8)
                else: image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                image = self.transform(image=image)['image']
                images_list.append(image)

        images_tensor = torch.stack(images_list, dim=0)
        target = [Config.CHAR2IDX[c] for c in label if c in Config.CHAR2IDX]
        if len(target) == 0: target = [0]

        return images_tensor, torch.tensor(target, dtype=torch.long), len(target), label

    @staticmethod
    def collate_fn(batch):
        images, targets, target_lengths, labels_text = zip(*batch)
        images = torch.stack(images, 0)
        targets = torch.cat(targets)
        target_lengths = torch.tensor(target_lengths, dtype=torch.long)
        return images, targets, target_lengths, labels_text

## 5. Model Architecture
Define the Attention Fusion module and the main CRNN model.

In [ ]:
class AttentionFusion(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.score_net = nn.Sequential(
            nn.Conv2d(channels, channels // 8, kernel_size=1),
            nn.ReLU(True),
            nn.Conv2d(channels // 8, 1, kernel_size=1)
        )
    def forward(self, x):
        b_frames, c, h, w = x.size()
        b_size = b_frames // 5
        x_view = x.view(b_size, 5, c, w)
        scores = self.score_net(x).view(b_size, 5, 1, w)
        return torch.sum(x_view * F.softmax(scores, dim=1), dim=1)

class MultiFrameCRNN(nn.Module):
    def __init__(self, num_classes, hidden_size=256):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 2), (2, 1), (0, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 2), (2, 1), (0, 1)),
            nn.Conv2d(512, 512, 2, 1, 0), nn.BatchNorm2d(512), nn.ReLU(True)
        )
        self.fusion = AttentionFusion(channels=512)
        self.rnn = nn.Sequential(nn.LSTM(512, hidden_size, bidirectional=True, batch_first=True, num_layers=2, dropout=0.25))
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        b, t, c, h, w = x.size()
        x = x.view(b * t, c, h, w)
        feat = self.cnn(x)
        fused = self.fusion(feat)
        out = self.fc(self.rnn(fused.permute(0, 2, 1))[0])
        return out.log_softmax(2)

## 6. Utility Functions
Helper function to decode model predictions into text.

In [ ]:
def decode_predictions(preds, idx2char):
    result_list = []
    for p in preds:
        pred_str = ""
        last_char = 0
        for char_idx in p:
            c = char_idx.item()
            if c != 0 and c != last_char: pred_str += idx2char[c]
            last_char = c
        result_list.append(pred_str)
    return result_list

## 7. Training Pipeline
The main training loop, including validation and testing.

In [ ]:
def train_pipeline():
    seed_everything(Config.SEED)
    print(f"🚀 TRAINING START | Device: {Config.DEVICE}")

    if not os.path.exists(Config.DATA_ROOT):
        print("❌ LỖI: Sai đường dẫn DATA_ROOT")
        return

    train_ds = AdvancedMultiFrameDataset(Config.DATA_ROOT, mode='train')
    val_ds = AdvancedMultiFrameDataset(Config.DATA_ROOT, mode='val')
    test_ds = AdvancedMultiFrameDataset(Config.DATA_ROOT, mode='test')

    if len(train_ds) == 0:
        print("❌ Dataset Train rỗng!")
        return

    train_loader = DataLoader(train_ds, batch_size=Config.BATCH_SIZE, shuffle=True,
                               collate_fn=AdvancedMultiFrameDataset.collate_fn, num_workers=10, pin_memory=True)

    if len(val_ds) > 0:
        val_loader = DataLoader(val_ds, batch_size=Config.BATCH_SIZE, shuffle=False,
                                collate_fn=AdvancedMultiFrameDataset.collate_fn, num_workers=10, pin_memory=True)
    else:
        print("⚠️ CẢNH BÁO: Validation Set rỗng. Sẽ bỏ qua bước validate.")
        val_loader = None

    if len(test_ds) > 0:
        test_loader = DataLoader(test_ds, batch_size=Config.BATCH_SIZE, shuffle=False,
                                 collate_fn=AdvancedMultiFrameDataset.collate_fn, num_workers=10, pin_memory=True)
    else:
        print("⚠️ CẢNH BÁO: Test Set rỗng.")
        test_loader = None

    model = MultiFrameCRNN(num_classes=Config.NUM_CLASSES).to(Config.DEVICE)
    criterion = nn.CTCLoss(blank=0, zero_infinity=True)
    optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=Config.LEARNING_RATE,
                                              steps_per_epoch=len(train_loader), epochs=Config.EPOCHS)
    scaler = GradScaler()

    best_acc = 0.0
    for epoch in range(Config.EPOCHS):
        model.train()
        epoch_loss = 0

        pbar = tqdm(train_loader, desc=f"Ep {epoch+1}/{Config.EPOCHS}")
        for images, targets, target_lengths, _ in pbar:
            images = images.to(Config.DEVICE)
            targets = targets.to(Config.DEVICE)

            optimizer.zero_grad(set_to_none=True)

            with autocast('cuda'):
                preds = model(images)
                preds_permuted = preds.permute(1, 0, 2)
                input_lengths = torch.full(size=(images.size(0),), fill_value=preds.size(1), dtype=torch.long)
                loss = criterion(preds_permuted, targets, input_lengths, target_lengths)

            scaler_scale_before = scaler.get_scale()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # Chỉ step scheduler nếu optimizer đã step (scale không bị giảm/reset)
            # Điều này fix warning "Detected call of lr_scheduler.step() before optimizer.step()"
            if scaler.get_scale() >= scaler_scale_before:
                scheduler.step()

            epoch_loss += loss.item()
            pbar.set_postfix({'loss': loss.item(), 'lr': scheduler.get_last_lr()[0]})

        avg_train_loss = epoch_loss / len(train_loader)

        val_acc = 0
        avg_val_loss = 0

        if val_loader:
            model.eval()
            val_loss = 0
            total_correct = 0
            total_samples = 0

            with torch.no_grad():
                for images, targets, target_lengths, labels_text in val_loader:
                    images = images.to(Config.DEVICE)
                    targets = targets.to(Config.DEVICE)
                    preds = model(images)

                    loss = criterion(preds.permute(1, 0, 2), targets,
                                     torch.full((images.size(0),), preds.size(1), dtype=torch.long), target_lengths)
                    val_loss += loss.item()

                    decoded = decode_predictions(torch.argmax(preds, dim=2), Config.IDX2CHAR)
                    for i in range(len(labels_text)):
                        if decoded[i] == labels_text[i]:
                            total_correct += 1
                    total_samples += len(labels_text)

            avg_val_loss = val_loss / len(val_loader)
            val_acc = (total_correct / total_samples) * 100 if total_samples > 0 else 0

        print(f"Result: Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print(f" -> ⭐ Saved Best Model! ({val_acc:.2f}%)")

    # ==========================================
    # INFERENCE ON TEST SET
    # ==========================================
    print("\n" + "="*60)
    print("🧪 EVALUATING ON TEST SET...")
    print("="*60)

    if test_loader and os.path.exists("best_model.pth"):
        # Load best model
        model.load_state_dict(torch.load("best_model.pth", weights_only=True))
        model.eval()

        test_correct = 0
        test_total = 0
        test_char_correct = 0
        test_char_total = 0

        results = []  # Lưu kết quả để phân tích

        with torch.no_grad():
            for images, targets, target_lengths, labels_text in tqdm(test_loader, desc="Testing"):
                images = images.to(Config.DEVICE)
                preds = model(images)
                decoded = decode_predictions(torch.argmax(preds, dim=2), Config.IDX2CHAR)

                for i in range(len(labels_text)):
                    gt = labels_text[i]
                    pred = decoded[i]

                    # Exact match accuracy
                    if pred == gt:
                        test_correct += 1
                    test_total += 1

                    # Character-level accuracy
                    for j in range(max(len(gt), len(pred))):
                        test_char_total += 1
                        if j < len(gt) and j < len(pred) and gt[j] == pred[j]:
                            test_char_correct += 1

                    # Lưu một số kết quả sai để debug
                    if pred != gt and len(results) < 20:
                        results.append({'gt': gt, 'pred': pred})

        test_acc = (test_correct / test_total) * 100 if test_total > 0 else 0
        char_acc = (test_char_correct / test_char_total) * 100 if test_char_total > 0 else 0

        print(f"\n📊 TEST RESULTS:")
        print(f"   • Exact Match Accuracy: {test_acc:.2f}% ({test_correct}/{test_total})")
        print(f"   • Character Accuracy:   {char_acc:.2f}%")
        print(f"   • Best Val Accuracy:    {best_acc:.2f}%")

        if results:
            print(f"\n🔍 Sample Errors (first 10):")
            for i, r in enumerate(results[:10]):
                print(f"   {i+1}. GT: '{r['gt']}' | Pred: '{r['pred']}'")

        # Lưu kết quả vào file
        test_results = {
            'test_accuracy': test_acc,
            'char_accuracy': char_acc,
            'best_val_accuracy': best_acc,
            'total_samples': test_total,
            'correct_samples': test_correct,
            'sample_errors': results
        }
        with open('test_results.json', 'w') as f:
            json.dump(test_results, f, indent=2)
        print(f"\n💾 Results saved to 'test_results.json'")

    else:
        if not test_loader:
            print("❌ Test loader không có dữ liệu!")
        if not os.path.exists("best_model.pth"):
            print("❌ Không tìm thấy best_model.pth!")

## 8. Run Training
Execute the training pipeline.

In [ ]:
if __name__ == "__main__":
    train_pipeline()

🔒 Đã cố định Seed: 42
🚀 TRAINING START | Device: cuda
[TRAIN] Scanning: /kaggle/input/icpr2026/train


/tmp/ipykernel_20/3928404161.py:78: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),


⚠️ Creating new split: 2000 val, 2000 test...
✅ Saved splits: val=2000, test=2000, train=16000
[TRAIN] Loaded 16000 tracks.


Indexing train: 100%|██████████| 16000/16000 [01:42<00:00, 155.76it/s]


[VAL] Scanning: /kaggle/input/icpr2026/train
📂 Loading splits from 'val_tracks.json' and 'test_tracks.json'...
[VAL] Loaded 2000 tracks.


...

Result: Train Loss: 0.0410 | Val Loss: 0.3099 | Val Acc: 73.45%


Ep 50/50: 100%|██████████| 250/250 [00:57<00:00,  4.35it/s, loss=0.0205, lr=4.29e-9]


Result: Train Loss: 0.0403 | Val Loss: 0.3097 | Val Acc: 73.35%

🧪 EVALUATING ON TEST SET...


Testing: 100%|██████████| 32/32 [00:07<00:00,  4.14it/s]


📊 TEST RESULTS:
   • Exact Match Accuracy: 73.20% (1464/2000)
   • Character Accuracy:   92.59%
   • Best Val Accuracy:    73.50%

🔍 Sample Errors (first 10):
   1. GT: 'POC2067' | Pred: 'POC2087'
   2. GT: 'AVX8286' | Pred: 'AVR8088'
   3. GT: 'AZV2616' | Pred: 'AZY2616'
   4. GT: 'AXY9931' | Pred: 'AVV9881'
   5. GT: 'AZO1492' | Pred: 'AZQ1492'
   6. GT: 'ATB9207' | Pred: 'ATB3559'
   7. GT: 'ARE5453' | Pred: 'ARE5457'
   8. GT: 'AXQ2352' | Pred: 'AYS2952'
   9. GT: 'AVJ7030' | Pred: 'AVJ7830'
   10. GT: 'BBL0621' | Pred: 'BBE0621'

💾 Results saved to 'test_results.json'
